In [1]:
import pandas as pd
!pip install fastparquet
data = pd.read_parquet("Flight Summary - North Atlantic - 2019.pq")

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
DEPRECATION: sqlalchemy-mate 1.4.28.3 has a non-standard dependency specifier sqlalchemy>=1.4.1<2.0.0. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of sqlalchemy-mate or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [3]:
data.head()

,ICAO Aircraft type designator,First known waypoint time (UTC),Last known waypoint time (UTC),Direction,Total flight dist (km),Total fuel burn (kg),Mean overall propulsion efficiency,Aircraft mass mean (kg),Mean nvPM number EI (kg-1),Initial contrail length (km),...,"Ice vol mean radius, Mean (um)","Tau contrail, Mean","Tau cirrus, Mean",RF SW (W m-2),RF LW (W m-2),RF Net (W m-2),Total contrail EF (J),SDR mean (W m-2),RSR mean (W m-2),OLR mean (W m-2)
0,B772,2019-01-01 02:17:49,2019-01-01 05:18:26,E,2633.994219,23991.530999,0.311201,228553.082349,4.920940e+14,1415.515126,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
1,B763,2019-01-01 02:19:06,2019-01-01 05:27:55,E,2670.997754,13886.660380,0.325782,150812.984413,1.425808e+15,1701.125144,...,3.716948,0.090483,0.001725,-0.565425,4.231059,3.665634,1.515245e+14,36.862724,15.583469,238.200396
2,B763,2019-01-01 02:21:40,2019-01-01 05:11:34,E,2388.740516,13397.610853,0.304915,150872.338643,1.172578e+15,2388.740516,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
3,B744,2019-01-01 02:16:31,2019-01-01 05:44:38,E,3039.310160,38210.558395,0.317271,311350.241271,5.589923e+14,2216.876721,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000
4,B789,2019-01-01 03:15:00,2019-01-01 04:30:15,E,1092.545055,9686.423444,0.282729,212017.428561,8.404441e+14,1092.545055,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000


In [4]:
# We set aside our target variable in this case - 1 if a a persisent contrail has a length > 0 , 0 otherwise

data['PersistentContrail_Formed'] = data['Persistent contrail length (km)'].apply(lambda x: 1 if x > 0 else 0)

In [5]:
# Next we do some feature engineering to deduce temporal characteristics about a flight given the waypoint times

# Convert the string columns to datetime
data['First known waypoint time (UTC)'] = pd.to_datetime(data['First known waypoint time (UTC)'])
data['Last known waypoint time (UTC)'] = pd.to_datetime(data['Last known waypoint time (UTC)'])

# Compute flight duration between waypoints, Time of Day and Month
data['Flight Duration'] = (data['Last known waypoint time (UTC)'] - data['First known waypoint time (UTC)']).dt.total_seconds()
data['Time of Day'] = data['First known waypoint time (UTC)'].dt.hour
data['Month'] = data['First known waypoint time (UTC)'].dt.month

With our list of features in hand - we need to understand which of them are most imporant in understanding our target variable - in this case, persistent contrail formation. The correlation matrix we used in our EDA highlights linear relationships but we need to understand the possibilty of nonlinear relationships as well. 

Our input feature space should be a list of features that have the highest linear and/or nonlinear relationships with our data. That will inform us which model is most effective in forecasting our target.

#### Linear Feature Analysis

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

numerical_features = data.select_dtypes(include=['float64', 'int64'])
# numerical_features.hist(bins=50, figsize=(20,15))

# correlation = numerical_features.corr()
# plt.figure(figsize=(15, 10))
# sns.heatmap(correlation, annot=True, cmap='coolwarm')
# plt.title("Correlation Matrix")
# plt.show()

#### Non-linear Feature Analysis

In [8]:
exclude_features = [
    'Persistent contrail length (km)',
    'Contrail altitude initial, Mean (ft)',
    'Contrail altitude lifetime, Mean (ft)',
    'Contrail age, Mean (h)',
    'Contrail age, Max (h)',
    'Initial contrail length (km)'
]

selected_features = [feat for feat in numerical_features.columns if feat not in exclude_features]

#### Determining spearman correlations

In [9]:
from scipy.stats import spearmanr

spearman_correlations = {}
for feature in selected_features:
    corr, _ = spearmanr(data[feature], data['PersistentContrail_Formed'])
    spearman_correlations[feature] = corr

# Display Spearman correlations sorted by absolute value
sorted_spearman = sorted(spearman_correlations.items(), key=lambda x: abs(x[1]), reverse=True)
for feature, corr in sorted_spearman:
    print(f"{feature}: {corr:.4f}")

PersistentContrail_Formed: 1.0000
OLR mean (W m-2): 0.8995
Tau contrail, Mean: 0.8995
Ice number initial, Stdev (m-1): 0.8995
Temp initial, Stdev (K): 0.8995
Ice number lifetime, Mean (m-1): 0.8995
Ice vol mean radius, Mean (um): 0.8995
RHi initial, Mean: 0.8995
RHi lifetime, Mean: 0.8995
Temp SAC, Stdev (K): 0.8995
Ice number initial, Mean (m-1): 0.8995
Temp initial, Mean (K): 0.8995
RHi initial, Stdev: 0.8995
Temp SAC, Mean (K): 0.8995
RF LW (W m-2): 0.8995
Tau cirrus, Mean: 0.8787
RSR mean (W m-2): 0.8399
SDR mean (W m-2): 0.7847
RF SW (W m-2): -0.7777
Total contrail EF (J): 0.5313
RF Net (W m-2): 0.3808
Flight Duration: 0.1746
Total fuel burn (kg): 0.1636
Total flight dist (km): 0.1577
Mean nvPM number EI (kg-1): -0.1339
Aircraft mass mean (kg): 0.0884
Mean overall propulsion efficiency: 0.0641
Month: -0.0575
Time of Day: -0.0135


#### Spearman feature analysis

#### Determining mutual information scores

In [11]:
from sklearn.feature_selection import mutual_info_regression

# Compute mutual information scores
mi_scores = mutual_info_regression(data[selected_features], data['PersistentContrail_Formed'])

# Display mutual information scores
for feature, score in sorted(zip(selected_features, mi_scores), key=lambda x: x[1], reverse=True):
    print(f"{feature}: {score:.4f}")

OLR mean (W m-2): 0.6875
Ice number lifetime, Mean (m-1): 0.6871
PersistentContrail_Formed: 0.6870
RHi initial, Stdev: 0.6869
Temp initial, Mean (K): 0.6864
RF Net (W m-2): 0.6861
Temp initial, Stdev (K): 0.6860
Temp SAC, Mean (K): 0.6859
RHi lifetime, Mean: 0.6858
Ice number initial, Mean (m-1): 0.6857
Ice number initial, Stdev (m-1): 0.6856
RF LW (W m-2): 0.6855
Ice vol mean radius, Mean (um): 0.6854
Tau contrail, Mean: 0.6854
RHi initial, Mean: 0.6853
Temp SAC, Stdev (K): 0.6848
Total contrail EF (J): 0.6286
Tau cirrus, Mean: 0.6190
RSR mean (W m-2): 0.5357
SDR mean (W m-2): 0.4477
RF SW (W m-2): 0.4386
Aircraft mass mean (kg): 0.0414
Total flight dist (km): 0.0390
Mean nvPM number EI (kg-1): 0.0265
Total fuel burn (kg): 0.0240
Flight Duration: 0.0228
Month: 0.0089
Mean overall propulsion efficiency: 0.0084
Time of Day: 0.0021


#### Determining mutual information scores 

#### Understanding multicollinearity

In [41]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant

# Add a constant to the model (i.e., bias or intercept)
df_const = add_constant(data[selected_features])

# Calculate VIF for each feature
vif = pd.DataFrame()
vif["Variable"] = df_const.columns
vif["VIF"] = [variance_inflation_factor(df_const.values, i) for i in range(df_const.shape[1])]

print(vif)

/Users/sean/opt/anaconda3/lib/python3.9/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  x = pd.concat(x[::order], 1)
/Users/sean/opt/anaconda3/lib/python3.9/site-packages/statsmodels/regression/linear_model.py:1715: RuntimeWarning: divide by zero encountered in double_scalars
  return 1 - self.ssr/self.centered_tss


                              Variable            VIF
0                                const       0.000000
1               Total flight dist (km)      10.953208
2                 Total fuel burn (kg)       6.372843
3   Mean overall propulsion efficiency       0.847503
4              Aircraft mass mean (kg)       4.518622
5           Mean nvPM number EI (kg-1)       1.559062
6         Initial contrail length (km)       1.561741
7                    RHi initial, Mean     113.044371
8                   RHi initial, Stdev       4.859674
9                   RHi lifetime, Mean     282.667968
10              Temp initial, Mean (K)    2736.647694
11             Temp initial, Stdev (K)       1.674260
12                  Temp SAC, Mean (K)    2674.739567
13                 Temp SAC, Stdev (K)       2.674064
14      Ice number initial, Mean (m-1)      14.434839
15     Ice number initial, Stdev (m-1)       4.338855
16     Ice number lifetime, Mean (m-1)      12.475935
17      Ice vol mean radius,

#### Train test split + PCA 

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix

X = data.drop('PersistentContrail_Formed', axis=1)
y = data['PersistentContrail_Formed']

X_encoded = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)